In [365]:
import pandas as pd
import numpy as np

In [366]:
df = pd.read_csv("measurements.csv", sep=',')
df.head()

,distance,consume,speed,temp_inside,temp_outside,specials,gas_type,AC,rain,sun,refill liters,refill gas
0,28,5,26,"21,5",12,NaN,E10,0,0,0,45,E10
1,12,"4,2",30,"21,5",13,NaN,E10,0,0,0,NaN,NaN
2,"11,2","5,5",38,"21,5",15,NaN,E10,0,0,0,NaN,NaN
3,"12,9","3,9",36,"21,5",14,NaN,E10,0,0,0,NaN,NaN
4,"18,5","4,5",46,"21,5",15,NaN,E10,0,0,0,NaN,NaN


In [367]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 12 columns):
distance         388 non-null object
consume          388 non-null object
speed            388 non-null int64
temp_inside      376 non-null object
temp_outside     388 non-null int64
specials         93 non-null object
gas_type         388 non-null object
AC               388 non-null int64
rain             388 non-null int64
sun              388 non-null int64
refill liters    13 non-null object
refill gas       13 non-null object
dtypes: int64(5), object(7)
memory usage: 36.5+ KB


In [368]:
#Afficher le nombre des lignes et des colones
df.shape

(388, 12)

In [369]:
#Afficher les moyennes et les écarts-types(std) de tous les features quantitatives
df.groupby('gas_type')['speed', 'temp_outside', 'AC'].describe()

speed                                                      \
          count       mean        std   min   25%   50%   75%   max   
gas_type                                                              
E10       160.0  43.506250  14.077949  14.0  35.0  42.0  51.0  88.0   
SP98      228.0  40.820175  13.170122  16.0  32.0  39.5  48.0  90.0   

         temp_outside            ...                  AC                      \
                count      mean  ...    75%   max  count      mean       std   
gas_type                         ...                                           
E10             160.0  10.11875  ...  14.25  27.0  160.0  0.043750  0.205181   
SP98            228.0  12.22807  ...  17.00  31.0  228.0  0.100877  0.301829   

                                   
          min  25%  50%  75%  max  
gas_type                           
E10       0.0  0.0  0.0  0.0  1.0  
SP98      0.0  0.0  0.0  0.0  1.0  

[2 rows x 24 columns]

In [370]:
for i in df.columns:
    print(i, "     ", df[i].dtype)

distance       object
consume       object
speed       int64
temp_inside       object
temp_outside       int64
specials       object
gas_type       object
AC       int64
rain       int64
sun       int64
refill liters       object
refill gas       object


In [371]:
#Les variables qui contiennent les valeurs manquantes
def features_nan_detect():
    for i in df.columns:
        x=df[i].isna().sum()
        if x != 0:
            print(i, "---Nombre", x)
            print(i, "---Proportion", x/388 * 100)

In [372]:
features_nan_detect()

temp_inside ---Nombre 12
temp_inside ---Proportion 3.0927835051546393
specials ---Nombre 295
specials ---Proportion 76.03092783505154
refill liters ---Nombre 375
refill liters ---Proportion 96.64948453608247
refill gas ---Nombre 375
refill gas ---Proportion 96.64948453608247


In [373]:
#Replacer les , par . afin de les convertir au float
df['distance'] = df['distance'].str.replace(',', '.')
#Changer le type des données de float à str, car le fit ne fonctionne pas avec les valeurs str
df['distance'] = df['distance'].astype(float)

In [374]:
df['consume'] = df['consume'].str.replace(',', '.')
df['consume'] = df['consume'].astype(float)
df['consume'].dtype

dtype('float64')

In [375]:
df['temp_inside'] = df['temp_inside'].str.replace(',', '.')
#df['temp_inside'] = df['temp_inside'].astype(float)
#df['temp_inside'].dtype

In [376]:
#Remplacer les valeurs Nan par le mode ou bien la médiane selon le type des données
from scipy.stats import mode

In [377]:
def replace_nan(colon):
    for index, row in df.iterrows():
        if pd.isna(row[colon]):
            t = df[colon].dtype
            if t in ['float64', 'int64']:
                df.loc[index, colon] = df[colon].median()
            else :
                df.loc[index, colon] = df[colon].mode().iloc[0]

In [378]:
def replace_nan_features(features_tabl):
    for i in features_tabl:
        replace_nan(i)

In [379]:
features_nan = ['temp_inside', 'specials', 'refill liters', 'refill gas']
replace_nan_features(features_nan)

In [380]:
#It looks like sklearn requires the data shape of (row number, column number). If your data shape is (row number, ) 
#like (999, ), it does not work. By using numpy.reshape(), you should change the shape of the array to (999, 1), e.g. using
######## X et y ne auront pas même taille
#X = X.transpose()

In [381]:
features = ['distance', 'temp_inside', 'temp_outside', 'gas_type','speed', 'AC', 'rain', 'sun']
to_dumnify = ['gas_type']
#to_dumnify1 = ['gas_type']
X = df[features]
y = df['consume']
#X_data=np.matrix(df[features])

#Y_data=np.matrix(df['consume'])
X.head()

,distance,temp_inside,temp_outside,gas_type,speed,AC,rain,sun
0,28.0,21.5,12,E10,26,0,0,0
1,12.0,21.5,13,E10,30,0,0,0
2,11.2,21.5,15,E10,38,0,0,0
3,12.9,21.5,14,E10,36,0,0,0
4,18.5,21.5,15,E10,46,0,0,0


In [382]:
##ss = [to_dumnify, to_dumnify1]
X = pd.get_dummies(X, columns=to_dumnify)
X.head()

,distance,temp_inside,temp_outside,speed,AC,rain,sun,gas_type_E10,gas_type_SP98
0,28.0,21.5,12,26,0,0,0,1,0
1,12.0,21.5,13,30,0,0,0,1,0
2,11.2,21.5,15,38,0,0,0,1,0
3,12.9,21.5,14,36,0,0,0,1,0
4,18.5,21.5,15,46,0,0,0,1,0


In [383]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression

In [384]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [385]:
model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [386]:
print(model.coef_)

[ 0.00288489 -0.01968269 -0.04129555 -0.01440134  0.36459147  0.54206487
 -0.19215783 -0.02429213  0.02429213]


In [ ]:
#def best_features():
 #   features = ['']

In [387]:
#consommation, calcule pour chaque 5mitres le nombre de litres consommés


In [388]:
#Calcul du score 
y_predicted = model.predict(X_test)
prob = model.score(X_test, y_test)
prob

0.17856148487492995

In [389]:
#Calcul de RMSE
from sklearn.metrics import mean_squared_error

In [390]:
#combined rmse value
rmse = np.sqrt(mean_squared_error(y_test, y_predicted))
print(rmse)

1.0447009579866564


## Random Forest Model 

In [391]:
from sklearn.ensemble import RandomForestRegressor

In [396]:
rnd_model = RandomForestRegressor()
rnd_model.fit(X_train,y_train)

C:\Users\mdele\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [399]:
y_rnd_predicted = rnd_model.predict(X_test)
prob_rnd = rnd_model.score(X_test, y_test)
prob_rnd

0.5629236915144313

In [400]:
rmse_rnd = np.sqrt(mean_squared_error(y_test, y_rnd_predicted))
print(rmse_rnd)

0.7620493468454259


In [405]:
#RMSE function
def rmse(y, y_pred):
    n = len(y)
    return (1/n * np.sum((y - y_pred)**2))**0.5

In [406]:
print(rmse(y_test, y_rnd_predicted))

0.7620493468454259
